# Data Science Capstone Assignment week 3, part 1
### Part two from line 158

## installing the necessary packages

In [8]:
pip install beautifulsoup4


     |████████████████████████████████| 112kB 6.7MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install lxml

     |████████████████████████████████| 5.8MB 26.7MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [14]:
from bs4 import BeautifulSoup
import bs4 as bs
import urllib.request

## Scraping data

In [15]:
sauce = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs.BeautifulSoup(sauce, 'lxml')

In [16]:
table = soup.table
print(table)

<table class="wikitable sortable">
<tbody><tr>
<th>Postal code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park / Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor / Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park / Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern / Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</td></tr>
<tr>
<td>M4B
</td>
<td>Ea

## Inserting data from the online table into a list that can be read into the df

In [122]:
data = []
for tr in table_rows:  
    td = tr.find_all('td') #find all table data
    row = [i.text.strip() for i in td] #add text to a list
    for i in row:  
        data.append(row) #append text of each row to the data list
print(data)

[['M1A', 'Not assigned', ''], ['M1A', 'Not assigned', ''], ['M1A', 'Not assigned', ''], ['M2A', 'Not assigned', ''], ['M2A', 'Not assigned', ''], ['M2A', 'Not assigned', ''], ['M3A', 'North York', 'Parkwoods'], ['M3A', 'North York', 'Parkwoods'], ['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village'], ['M4A', 'North York', 'Victoria Village'], ['M4A', 'North York', 'Victoria Village'], ['M5A', 'Downtown Toronto', 'Regent Park / Harbourfront'], ['M5A', 'Downtown Toronto', 'Regent Park / Harbourfront'], ['M5A', 'Downtown Toronto', 'Regent Park / Harbourfront'], ['M6A', 'North York', 'Lawrence Manor / Lawrence Heights'], ['M6A', 'North York', 'Lawrence Manor / Lawrence Heights'], ['M6A', 'North York', 'Lawrence Manor / Lawrence Heights'], ['M7A', 'Downtown Toronto', "Queen's Park / Ontario Provincial Government"], ['M7A', 'Downtown Toronto', "Queen's Park / Ontario Provincial Government"], ['M7A', 'Downtown Toronto', "Queen's Park / Ontario Provincial Government"], 

In [123]:
import pandas as pd
from pandas import DataFrame

In [124]:
df = DataFrame(data, columns=['PostalCode','Borough','Neighbourhood']) #add columns
df.head() 

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,
1,M1A,Not assigned,
2,M1A,Not assigned,
3,M2A,Not assigned,
4,M2A,Not assigned,


# Remove unassigned boroughs

In [134]:
indexNames = df[df['Borough'] == 'Not assigned'].index 
df.drop(indexNames, inplace=True) #remove not assigned boroughs
df.head()

,PostalCode,Borough,Neighbourhood
6,M3A,North York,Parkwoods
7,M3A,North York,Parkwoods
8,M3A,North York,Parkwoods
9,M4A,North York,Victoria Village
10,M4A,North York,Victoria Village


# Merge Neighbourhoods

In [149]:
temp_df=df.groupby('PostalCode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighbourhood':'Neighbourhood_joined'},inplace=True)

In [150]:

df_merge = pd.merge(df, temp_df, on='PostalCode')

In [151]:

df_merge.drop(['Neighbourhood'],axis=1,inplace=True)

In [152]:
df_merge.drop_duplicates(inplace=True)

In [155]:

df_merge.rename(columns={'Neighbourhood_joined':'Neighbourhood'},inplace=True)

In [156]:
df_merge.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,"Parkwoods, Parkwoods, Parkwoods"
3,M4A,North York,"Victoria Village, Victoria Village, Victoria V..."
6,M5A,Downtown Toronto,"Regent Park / Harbourfront, Regent Park / Harb..."
9,M6A,North York,"Lawrence Manor / Lawrence Heights, Lawrence Ma..."
12,M7A,Downtown Toronto,"Queen's Park / Ontario Provincial Government, ..."


# Check shape of dataframe

In [141]:
df_merge.shape

(103, 3)

# Capstone Project Week 3 part 2

In [158]:

def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [159]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [161]:
geo_df.rename(columns={'Postal Code':'PostalCode'},inplace=True)
geo_merged = pd.merge(geo_df, df_merge, on='PostalCode')
geo_data=geo_merged[['PostalCode','Borough','Neighbourhood','Latitude','Longitude']]
geo_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern / Rouge, Malvern / Rouge, Malvern / Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill / Port Union / Highland Creek, Roug...",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood / Morningside / West Hill, Guildwood...",43.763573,-79.188711
3,M1G,Scarborough,"Woburn, Woburn, Woburn",43.770992,-79.216917
4,M1H,Scarborough,"Cedarbrae, Cedarbrae, Cedarbrae",43.773136,-79.239476


# Capstone project week 3 Part 3